# 02 – Image to Text 

In [36]:
import pytesseract
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import cv2

In [37]:
myurl = "https://www.pexels.com/de-de/foto/banner-internet-haus-vereinigte-staaten-von-amerika-5821295/"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                         'AppleWebKit/537.36 (KHTML, like Gecko) '
                         'Chrome/126.0.0.0 Safari/537.36'}

req = Request(myurl, headers=headers)
html = urlopen(req).read()
soup = BeautifulSoup(html, 'html.parser')

In [38]:
image_tag = soup.find('img', class_='PhotoZoom_image__2c8qM')

if image_tag and image_tag.has_attr('src'):
    image_url = image_tag['src']

# download it
img_data = requests.get(image_url).content
with open('pexels_image.jpg', 'wb') as handler:
    handler.write(img_data)

print("Image downloaded and saved as pexels_image.jpg")

Image downloaded and saved as pexels_image.jpg


In [39]:
img = cv2.imread("pexels_image.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

data = pytesseract.image_to_data(gray, output_type=Output.DICT)

# Group words by 'block_num' (each block is a paragraph or headline)
blocks = {}
for i, text in enumerate(data['text']):
    if text.strip():
        block_id = data['block_num'][i]
        if block_id not in blocks:
            blocks[block_id] = {'text': [], 'area': 0}
        blocks[block_id]['text'].append(text)

        w, h = data['width'][i], data['height'][i]
        blocks[block_id]['area'] += w * h

# Find the block with the largest total area
largest_block = max(blocks.values(), key=lambda b: b['area'])
headline = " ".join(largest_block['text'])

print("Detected headline:\n", headline.lower())

Detected headline:
 biden beats trump
